#  1.  Reinforcement Learning

This notebook introduces the fundamental Reinforcement Learning paradigm and algorithm.  Reinforcement Learning (RL) is a general framework for building computational agents that can, if trained properly, act intelligently in a well-defined but dynamic enviroment.  

This notebook contains an overview of the fundamentals of Reinforcement Learning, and in particular Q learning.  We first discuss the principals of RL for small problems, then describe how function approximators are integrated into the RL framework to tackle more challenging problems. 

## 1.1 Common RL problems

The two major areas where RL is currently applied are in automatic control (and robotics in particular) and in video / board game AI.  Here we introduce a variety of common RL problems that we will return to as we go deeper into details.

### Example 1:  Grid world: making a shortest-path finding AI

Grid world is a simplified version of what is called the shortest path problem - a problem often solved in video games (this is how enemy AI units find you in the game), mapping services (finding the shortest route from A to B), and robot path planning (e.g., for a cleaning robot) - get a user from its starting location to a desired destination in as few steps as possible.    

There are many algorithms specifically designed to solve just this task - the most popular being [Dijkstra’s and A* algorithms](http://www.redblobgames.com/pathfinding/a-star/introduction.html).  However here we will use the flexible RL framework as it too provides great results, and the (relative) simplicity of the problem will allow us to illustrate the more general RL learning process hopefully clearly.

In this toy problem we have a grid like the one illustrated below.  Each square tile is a location in the world.  Here the black square denotes the user, the green square the desired destination, and the blue squares impenetrable obstacles (the user cannot move to).  

The actions available to our user are to move one unit left, right, up, or down and it can move to any free square (here colored grey) any hazard square (think hot coals! shown in orange) and the goal (colored green).  If the user ever moves to the goal square the game is over.

In [1]:
from my_gridworld import my_gridworld
test = my_gridworld(grid_size = 'small')
test.color_grid()

,,,,
,,,,
,,,,
,,,,


Remember - to see any of test's functions simply type    

test.function_name??   

in a Python cell.  Note the two qestion marks are necessary!  For example, activating the next Python cell will show you the contents of the color_grid function.

In [38]:
# show the contents of the color_grid function
test.color_grid??

###  Example 2:  Making an AI that can play chess

Another problem RL can be applied to is making an intelligent agent to play (and win) the game of chess.

<img src="images/chess.png",width=250,height=250>

### Example 3:   An AI that can balanace a pole on a moving cart (a.k.a. the inverted pendulum)

A classic toy control problem  - teach an agent how to balance a pole on a moving cart.  The pole is free to rotate about an axis on the cart, with the cart on a track so that it may be moved left and right - affecting the location of the pole.  The pole feels the force of Earth's gravity, and so if unbalanced will fall to the ground.  

<img src="images/cartpole.png",width=200,height=200>

This problem is a prototype for more general problems in mechanical control and robotics.

## 1.2  The fundamental components of an RL problem

Lets define the modeling components of a standard Reinforcement Learning problem - afterwards we will return to our example problems and describe the specific instances of these components for each example.  

**There are *four* main components to the RL model: states, features, actions, and rewards.**  We will talk about the first 3 now, and deal with features in the next section.  So the first three main concepts are - at a high level:

1\.    A **state** is a specific configuration of all objects in a problem's enviroment.  A **state space** is the collection of all possible states this enviroment is capable of producing.  The state space is defined by the problem we aim to solve.

- In *grid world* any state consists of one piece of information: the current location of the player (the black square).  The state space is all possible locations on the board where the player can go.


- In the chess problem a state is any (legal) configuration of the white and black pieces on the board (and some pieces may be missing).  The state space is the set of all possible configurations of chess pieces on the board.  The state space is defined by the problem we aim to solve.


- In the cart-pole problem a state is a complete set of information about the cart and pole's position.  This includes: the cart position, the cart velocity, the angle of the pole measured as its deviation from the vertical position, and the angular velocity of the pole.  While these are technically continuous values, in practice they are finelly discretized.  The state does **not** include technical / physical information about the enviroment e.g., the fact that gravity exists, its precise force, etc.,


2\.  The agent can then take an **action** - the set of actions is either determined by us or by the problem environment.  The **action space** is the total set of such available actions.  


- For grid world, a toy problem, we can choose a set of actions.  For example we could decide that the agent can only move the black square adjacently one unit up/down/left/right or keep it still.  Alternatively, we could allow the agent to move the black square diagonally as well.


- In the chess problem an action is a legal move of any of its current pieces on the board.


- In the cart pole problem - as in robotics applications as well - the range of actions is completely defined by the available range of motions of the machine being directly controlled.  In cart pole the agent directly controls the cart, which in 2-dimensions can only move left or right.


3\.  Once this action is taken the agent receives a **reward** - or a signal that the agent has performed a good or bad action based on our desired goal.  We (humans) decide on the reward structure (in order to induce our agent to accomplish our desired goal).  We want a reward for a given action to be **larger** for those actions that get us closer to accomplishing our goal (and less for those actions which do not), or vice versa.  To reiterate: this is the (only) way we communicate our desired goal to the agent.

- For grid world we can assign a negative value like -1 to all actions (one unit movement) which leads to a non-goal state, and a large positive number like 1000 to actions leading to the **goal state** itself.  However these exact numbers are arbitrary, so long as a negative reward is given to actions not directly leading to the goal (and a positive reward is given when the goal square is reached).
 
 
- For the chess problem one reward structure to induce our agent to learn how to win could be as follows: any move made that does not lead to the **goal state** (checkmating the opponent) recieves -1 reward, a move that sucessfully check mates the oponnent receieves a reward of 10000


- For the cart-pole problem one common choice of reward structure - at every state at which the angle of the pole is above a certain threshold the reward is 1, otherwise it is 0.

##  1.3  How the four RL pieces fit together

These four pieces connect together to create a feedback system that - when properly engineered - allow us to train an AI agent to accomplish a desired goal via maximization of the surrogate rewards.  In order to derive the RL algorithms that allow for this we need to first represent each of the components algebraically.

### Problem solving in steps and RL component notation

For now suppose a for a given problem that there are only $N$ possible states and $M$ available actions.  

- In the grid world problem the number of states is finite and equal to the number of places the player can travel.


- In the chess problem the number of states is equal to the total number of possible configurations of the board, while the number of actions is equal to the number of legal moves available to the pieces on the board.


- In the cart pole problem the number of states is equal to the number of configurations of the 4 discreteized enviromental descriptors, and there are only three actions: move the cart left, right, or keep it still


For a given problem we will denote the state and actions spaces as follows

- Denote the set of all states of the system  $S=\left\{ \sigma_{1},\sigma_{2},...,\sigma_{N}\right\} $ 


- Denote the set of all actions available  $A=\left\{ \alpha_{1},\alpha_{2},...,\alpha_{M}\right\} $


Notice that if we ourselves were to solve any of the examples given so far we would do so sequentially, in discrete steps.  This is precisely how the agent solves these problems as well.  

- In grid world we / the agent move the player one square at a time.  Beginning at a state (location) an action is taken to move the player to a new state.  For taking this action and moving to the new state the agent receives a reward.  This reward is either positive or negative based on whether - in the long run - this helped or hindered it achieivng the goal (reaching the green square).


- In chess we / the agent moves one piece at a time.  Beginning at a state (a specific configuration of the board) an action is taken and a piece on the board is moved, a new state of the system.  For taking this action and moving to the new state the agent receives a reward.  This reward is either positive or negative based on whether - in the long run - this helped or hindered it achieivng the goal (check mating the opponent).


- In cart pole we / the agent keep the cart still or move it with a fixed force left or right.  Beginning at a state (a specific configuration of the 4 system descriptors) an action is taken and the cart is moved, a new state of the system arises.  For taking this action and moving to the new state the agent receives a reward.  This reward is either positive or negative based on whether - in the long run - this helped or hindered it achieivng the goal (keeping the pole in the air as long as possible).


Some notation to describe what occurs at the $k^{th}$ step in solving a problem: at this step an agent begins at a state $s_{k-1} \in S$, takes an action $a_k \in A$ that moves the system to a state $s_k \in S$.

A note about this notation: be careful not to confuse the $s$'s with the $\sigma$'s, and the $a$'s with the $\alpha$'s.  The notation $s_{k-1}$ is a *variable* denoting the state at which the $k^{th}$ step of the procedure begins, and so can be any of the possible realized states $S=\left\{ \sigma_{1},\sigma_{2},...,\sigma_{N}\right\} $.  Likewise the notation $a_k$ is a variable denoting the action taken at the $k^{th}$ step, which can be any of the possible actions $A=\left\{ \alpha_{1},\alpha_{2},...,\alpha_{M}\right\} $.

Notice too a key part of each of these descriptions when it comes to an agent solving a given problem: the mechanism by which an agent learns the best action to take in a given state is the reward it recieves.  This reward is positive or negative based on how each action helps or hinders its accomplishment of the goal.

We need notation for the reward an agent recieves at the $k^{th}$ step as well: call this $r_k$.  This is a function of the initial state at this step the action taken - i.e., $r_k = r_k(s_{k-1},a_k)$  

### A brief note about stochastic problems

Not all problems are 'deterministic' in nature, such as those considered here.  That is one realized action $a_k$ at a given state $s_{k-1}$ always leads to the same next state $s_k$.  In *stochastic* problems a given action at a state may lead to different conclusions.  For example robotics RL-based problems are often stochastic in nature - e.g., a robot may perform a given action - like accelerate forward one unit of thrust - at the a state and reach different outcomes due to things like inconsistencies in the application of this action, sensor issues, or friction.

Almost the same modeling we used above captures this variability for such stochastic problems, with the main difference being that the reward function must also necessarily be a function of the state $s_k$ as well $a_k$ and $s_{k-1}$.  We won't deal with this generality until we flush out all the details of the deterministic scenario first.

### Episodes of simulation

So in solving a problem an agent goes through a sequence of events at each step.  These are: start at a state, take an action, move to a new state, and recieve a corresponding reward.  The first three steps look like

**step 1:** start at state 0 $s_0$ --> take action 1 $a_1$ --> move to state 1 $s_1$ + recieve reward 1 $r_1$

**step 2:** start at state 1 $s_1$ --> take action 2 $a_2$--> move to state 2 $s_2$ + recieve reward 2 $r_2$

**step 3:** start at state 2 $s_2$ --> take action 3 $a_3$ --> move to state 3 $s_3$ + receive reward 3 $r_3$

**step 4:** ...

or in short the first three steps look like

($s_0$, $a_1$, $r_1$), ($s_1$, $a_2$, $r_2$), ($s_2$, $a_3$, $r_3$), ($s_3$,...

Taken together a sequence of such steps - ending either when a goal state is reached (as in the grid world or chess examples) or after a maximum number of iterations is completed (as in the cart pole example) - is referred to in RL jargon as an **episode**.  

What do we want the perfect epsiode of steps to look like when performed by a trained agent?  We want the agent to choose actions in order to maximize its long term reward - that is the sum of rewards over all the steps in the episode.

# 2  Training the agent by Q learning

So far we have set up the standard RL problems, defined its various components and introduced notation for each, and discussed how these components interact in each step of a problem solving process.  In this Section we will discuss a basic approach to training an agent to take the right actions to maximize its total rewards.

## 2.1  The fundamental and recursive RL cost function

How do we do this - train the agent to take the proper actions at each state to maximize its total reward?  First - as with all machine learning problems - we need to form a cost function.  

Lets take an arbitrary episode of steps beginning at state $s_0$ where we take action $a_1$

($s_0$, $a_1$, $r_1$), ($s_1$, $a_2$, $r_2$), ($s_2$, $a_3$, $r_3$), ($s_3$,...

After $a_1$ is taken want to determine the best sequence of remaining actions so that the sum of the rewards for these actions is as large as possible.

So denote by $Q(s_0,a_1)$ the *maximum total reward possible* if we begin at a state $s_0$, take the action $a_1$ bringing us to some state $s_1$, and then continue taking steps until goal is reached or a maximum number of steps is taken.  By definition this quantity is the sum of the realized reward $r_1$ (we recieve for taking action $a_1$ at state $s_0$) plus the largest possible rewards from all the preceeding steps.  

The only thing that is not 'maximal' about this $Q$ is the first reward $r_1$ - since there could be another action that provides a larger first one.  So then the maximum possible value of $Q$ at the state $s_0$ is achieved when we choose this best action at state $s_0$ 

$\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{0},\,a_{i}\right)$

With the above the $Q$ function can be defined *recursively* in terms of itself.  This is because as stated the quantity $Q(s_0,a_1)$ must be the sum of $r_1$ - the reward for taking the action $a_1$ at $s_0$ - plus the *maximum possible sum of rewards for all subsequent steps* $\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{1},\,a_{i}\right)$.

Writing this out algebraically (often referred to as **Bellman's equation** this is precisely

$Q\left(s_{0},\,a_{1}\right)=r_{1}+\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{1},\,a_{i}\right)$

And in fact this definition holds regardless of what state and action we begin with.  i.e., at the ${k-1}^th$ step $s_{k-1}$ if we take action $a_k$ we then have the relationship 

$Q\left(s_{k-1},\,a_{k}\right)=r_{k}+\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{k},\,a_{i}\right)$

## 2.2  Slight adjustment to dampen the effect of late actions

Often in practice one includes a parameter$\gamma \in [0,1]$ to regularize or dampens later rewards and prevent the affect of later actions to interfere with maximal actions in the present state $s_0$.  Adding this parameter we have a recursive definition of $Q$ as

$Q\left(s_{k-1},\,a_{k}\right)=r_{k}+\gamma\cdot\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{k},\,a_{i}\right)$

By scaling $\gamma$ up and down we can lessen the contribution of future rewards.  For example

- When we set $\gamma = 0$ then only the first reward remains $r_1$.   Maximizing $Q$ means we maximize the reward given by the first step of the process, for all states.  Our agent learns to take a 'greedy' approach to accomplishing our goal, at each state taking the next step that maximizes the next step reward only.


- When we set $\gamma = 1$ then we all the $\gamma$'s disappear and we have our original cost function. 

##  2.3  Computing Q in practice

How do practically compute the $Q$ function in practice using the recursive definition discussed above?  At first it might seem like this is impossible - a chicken and egg problem - how do we compute $Q$ if it is defined via itself?

The answer: by running a large number of episodes and updating $Q$ via the recursive definition as we go along.  In the most basic appoach we run each episode we and a random initial state, a random action, and repeat taking steps until a goal state is reached or maximum number of steps is taken.  At the $k^{th}$ step we are at a state $s_{k-1}$ and take a random action $a_k$ and update $Q(s_{k-1},a_k)$ using the recursive formula.

By running through a large number of episodes - and so through the gamat of all states and actions multiple - updating $Q$ at each step we essentially learn $Q$ by trial-and-error interactions with the enviroment.  How well our computations converge to the true $Q$ function depend on how well we sample the state / action spaces.

------
Initialize $Q$, choose value for $\gamma \in [0,1]$

**for** e = 1...E (the maximum number of episodes)
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Select a random initial state $s_0$ and $k=0$
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **while** end state not reached OR maximum iteration count not met

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;select a random action $a_k$ at our current state $s_{k-1}$ and move to $s_k$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;update $Q$ as 
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$Q\left(s_{k-1},\,a_{k}\right)=r_{k}+\gamma\cdot\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{k},\,a_{i}\right)$
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$k \longrightarrow k+1$

-----

This algorithm - called **Q-learning** - is performed for the grid world example in the following Python cell.

In [2]:
# train q learning and lets examine the matrix as it develops
test.qlearn(gamma = 0.8,hazard_penalty = -4,num_train_animate = 2)

continuing with remaining episodes without animation...
q-learning process complete


To expose the contents of the qlearn gridworld Q learning function activate the Python cell below.  You will see that it follows precisely the pseudo-code given above.

In [4]:
# activate this cell to see the Q learning algorithm used for gridworld above
test.qlearn??

## 2.4  Representing the Q function as a 2-dimensional matrix

While mathematically the input to $Q(s,a)$ can have many dimensions, being pairs of raw states and actions, the way we compute it via *Bellman's equation* / store it on a computer is via a 2-dimensional matrix representation.  

Remember we denote all the realized states as $S=\left\{ \sigma_{1},\sigma_{2},...,\sigma_{N}\right\} $, and likewise the set of all possible actions $ A=\left\{ \alpha_{1},\alpha_{2},...,\alpha_{M}\right\} $, then we represent $Q$ on as computer as the $N \times M$ matrix


$Q=\left[\begin{array}{cccc}
Q\left(\sigma_{0},\alpha_{0}\right) & Q\left(\sigma_{0},\alpha_{1}\right) & \cdots & Q\left(\sigma_{0},\alpha_{M}\right)\\
Q\left(\sigma_{1},\alpha_{0}\right) & \ddots &  & Q\left(\sigma_{1},\alpha_{1}\right)\\
\vdots &  & \ddots & \vdots\\
Q\left(\sigma_{N},\alpha_{0}\right) &  &  & Q\left(\sigma_{N},\alpha_{M}\right)
\end{array}\right]$

In other words, we represent $Q$ as a matrix indexed by all possible actions along its columns and all possible states along its rows.

In the next Python cell you can see the final Q matrix learned via the Q-learning algorithm for grid world.  The vertical axis is indexed by state location (e.g., the top left square is 00, the bottom right is 34) while the top labels are indexed by the available actions. 

In [8]:
# activate this cell to see the Q learning algorithm used for gridworld above
test.show_qmat()

       up   down   left  right
00 -4.463 -4.329 -4.463 -4.329
01 -4.329 -4.161 -4.463 -4.161
02 -4.161 -3.951 -4.329 -4.000
03 -1.000 -4.800 -4.161  0.000
04  0.000  0.000  0.000  0.000
10 -4.463 -4.161 -4.329 -4.161
11 -4.329 -3.951 -4.329 -3.951
12 -4.161 -3.689 -4.161 -4.800
13 -4.000 -5.440 -3.951 -1.000
14  0.000 -1.800 -4.800 -1.000
20 -4.329 -3.951 -4.161 -3.951
21 -4.161 -3.689 -4.161 -3.689
22 -3.951 -3.362 -3.951 -5.440
23 -4.800 -2.952 -3.689 -1.800
24 -1.000 -2.440 -5.440 -1.800
30 -4.161 -3.951 -3.951 -3.689
31 -3.951 -3.689 -3.951 -3.362
32 -3.689 -3.362 -3.689 -2.952
33 -5.440 -2.952 -3.362 -2.440
34 -1.800 -2.440 -2.952 -2.440


## 2.5  What is the best action per state given a learned Q function?

Given $Q(s_k,a)$ learned properly, how should the agent move at each state?  This is implicit in the recursive definition of $Q$ itself.  Since the action maximizing long term reward maximizing reward at state $s_k$ is given by 

$\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{k},\,a_{i}\right)$

the action $a_k$ providing this maximimum is given as the argument of this maximum

$a_{k}^{\star}=\underset{i=1...M}{\text{argmax}}\,\,Q\left(s_{k},\,a_{i}\right)$

(A note on jargon: this is often referred to as an **optimal policy** function.)  

So in sum - after learning $Q$ properly we can start at any initial state $s_0$ and use the optimal policy function to take actions that allow us to travel in a reward maximizing path of states until we reach the goal (or a maximum number of steps are taken).

In the next Python cell we illustrate how to use the optimal policy function with the grid world example (where we have already learned the proper $Q$ matrix in a previous Python cell).  You can initialize the agent at any square on the board (except barrier locations in blue), and activating the cell will animate its path (using the optimal policy function).

In [4]:
# run testing phase
starting_location = [0,0]
test.animate_testing_phase(starting_location)

,,,,
,,,,
,,,,
,,,,


Try several starting locations and run the above cell.

Notice how the agent always takes the shortest route to the goal while avoiding the hazards?  

### More tinkering in grid world

The second input parameter to the qlearn function for gridworld - called 'hazard_penalty' - is the penalty for stepping on a hazard (the orange squares).  If we lower this penalty value we can change the behavior of the fully trained agent.  i.e., we can make it cheaper for the agent to actually walk on a hazard square!  

Try adjusting the hazard_penalty parameter in the following Python cell.  Run the cell and then play with the subsequent testing phase cell where the final learned agent moves towards the goal.  

In [5]:
# train q learning and lets examine the matrix as it develops
test.qlearn(gamma = 0.8,hazard_penalty = -3,num_train_animate = 0)

continuing with remaining episodes without animation...
q-learning process complete


In [11]:
# run testing phase
starting_location = [0,0]
test.animate_testing_phase(starting_location)

,,,,
,,,,
,,,,
,,,,


The grid world arrangement we have been playing with is rather arbitrary - all we need is a player (blue square), goal (green square) and possibly hazards.  The Q learning algorithm will still work in precisely the manner detailed above, as will the testing phase with the learned agent.

Check out the next several Python cells - here we take a larger gridworld with a random assortment of hazards and follow the same procedure as before.

In [1]:
# activate this cell to make a much larger grid world example!
from my_gridworld import my_gridworld
test = my_gridworld(grid_size = 'large')
test.color_grid()

,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,


Next we perform Q learning for this example.  You will see that - in general - each episode of simulation takes a lot longer to complete because the world is much larger (and so reaching the goal by taking random steps takes much longer).  

In [2]:
# train q learning and lets examine the matrix as it develops
test.qlearn(gamma = 0.8,hazard_penalty = -3,num_train_animate = 2)

finished episode, animating next episode
continuing with remaining episodes without animation...
q-learning process complete


Next up is the testing phase where we set our trained agent loose to find the goal as cheaply as possible.

In [3]:
# run testing phase
starting_location = [9,0]
test.animate_testing_phase(starting_location)

,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,


# 3.  The importance of features

In detailing the pieces of a standard RL problem in the first Section we mentioned 3 components: states, actions, and rewards.  But there is an important $4^{th}$ component to an RL system we did not mention - features.  

In this Section we describe this final RL component and weave features into formal description in Section 2 that led to the Q learning scheme.

## 3.1  What are features?

Features, the $4^{th}$ RL component, play the same role in Reinforcement Learning as they do in machine learning more generally: they characterize input so that a proper relationship between input and output can be learned.

**Features** are pieces of information that we (humans) design and which characterize each state in the state space.  The agent uses the features from each state to make its decision on how to behave to achieve our goal.  Again, what sort of information to give to our agent **is completely up to us to decide.**  


-  For grid world a fine choice of features are simply the raw state data itself - that is the tuple (the horizontal / vertical coordinates) describing the location of the player.  Another choice: two features 1) the distance from the player to the goal (one feature) and 2) the distance from player to the nearest wall. 


- For the chess problem a natural choice is one set of 64 features, the $i^{th}$ feature indicating which (if any) piece is occupying the $i^{th}$ square of the board (there are 64 squares in total, hence 64 features).  However there are other choices we could make as well.


- For the cart-pole problem a natural set of features are just the four state descriptors (discretized properly).  


- A common choice of features for many Reinforcement Learning problems with very large state spaces are **mathematical bases** such as polynomial, Fourier, decision tree, or neural networks.  Often the input to these features are the raw (or slightly processed) states themsevles.  See Section 4 for further details on this topic.

## 3.2  Formally integrating features into the RL framework

Formally we can denote a set of features extracted from a state $s$ using function notation as $f(s)$.  Feature transformations in RL are often a set of abstract processing steps on the problem enviroment (as in the grid-world or chess examples) and need not solely be a function in the classical mathematical sense (i.e., $f (s) = \text{sin}(s)$ ).  

Also note - that typically $f(s)$ is **vector valued**.  For example, the natural choice of features in grid world is a tuple $f(s) = (~f_1(s),~f_2(s)~)$ where $f_1$ and $f_2$ denote the $x$ and $y$ coordinate of the player.  Likewise for the chess problem the natural set of features has length 64 i.e., $f(s) = (f_1(s),~f_2(s),...,~f_{64}(s)~)$ where the $i^{th}$ feature indicates which (if any) piece is currently in the $i^{th}$ square of the board.

An AI agent - and the Q learning process - learns based on the features extracted at each state.  This means that in general the input to the $Q$ function are not states and actions as $Q(s,a)$, but features and actions $Q(\text{f}(s),a)$.  $Q$ is still stored and computed as an $N \times M$ matrix, but is the features from each state that are used as input instead of the raw state itself (unless the features are chosen to be the state itself).


$Q=\left[\begin{array}{cccc}
Q\left(f\left(\sigma_{0}\right),\alpha_{0}\right) & Q\left(f\left(\sigma_{0}\right),\alpha_{1}\right) & \cdots & Q\left(f\left(\sigma_{0}\right),\alpha_{M}\right)\\
Q\left(f\left(\sigma_{1}\right),,\alpha_{0}\right) & \ddots &  & Q\left(f\left(\sigma_{1}\right),\alpha_{1}\right)\\
\vdots &  & \ddots & \vdots\\
Q\left(f\left(\sigma_{N}\right),,\alpha_{0}\right) &  &  & Q\left(f\left(\sigma_{N}\right),\alpha_{M}\right)
\end{array}\right]$

## 3.3  What features should you choose 

### Choosing features that provide the best information about your problem states

Many machine learning problems require that you know a substantial amount of information about a given domain in order to make a proper choice of features.  You obviously do not want features that represent your data incorrectly, as if you use such features no algorithm on earth can help you solve your problem.  In two class classification - for example - if you choose features that cause your classes to completely intermix you won't be able to perform your task very well.

This intuition carries over to the RL setting as well.  Generally speaking the same holds in the RL setting as well.  If you are working on a complex robotics problem your time spent engineering features could be substantial.  Analog to classification, in the RL setting you want features that sharply distinguish between *states* of your system.  If your features fail to accomplish this you will train a poorly performing agent.  And designing features that accomplish this goal can require serious reflection and work.  

For some RL problems - however - choosing reasonable features is fairly intuitive.  In the toy problems we discussed here - for example - choosing features was straight forward (sometimes we just took the raw state itself as the feature).  In other game related problems the choice can be more intuitive as well - but the more you understand the game the better the features you can devise.

### The information content of features vs cost of simulation

With typical machine learning problems you don't want your features to misrepresent your data (the worst case scenario), but you also don't want the to be uninformative (a milder problem).  Uninformative features don't mess up your data - e.g., in the two class classification scenario they don't mix up your two classes - but **the less informative your features are the more data you need to determine your system's underlying rule**.

The same holds for the RL scenario - with one important wrinkle: with RL problems you *create your own dataset* in learning the $Q$ function properly.  So you can get away with having fairly uninformative features if you have the computational power to compensate for them, and run a very large number of epislodes.  This is certainly possible with a problems like the chess or cart pole examples - we can simulate until the cows come home using e.g., a cloud server.  The cost of simulating is low.  This is not the case for many robotics applications however, where an episode must be enacted by a physical robot (hence limiting the number of episodes we can effectively run).


### Choosing features based on a product or research goal

Another question to consider when choosing features for RL is: what are your intentions?  

- Are they practical?  Do you want to produce a robot with some automatic control feature and sell it to paying customers?  Then you want invest heavily in resources to construct a set of features that will work quickly and effectively with your application.


- Are they research-focused?  Then your goal may be set by a curiousity of your own or an academic or industry establishment.  One academic question investigated a lot these days is to find a set of features you can effectively apply to solving a large set of visual problems (e.g., convolutional networks for training Atari AI).  These features are not very informative, but because computer power and storage is so great these days we can make these features work by running a very large number of episodes to learn $Q$ properly.

# 4.  Dealing with large state spaces

Notice one major fundamental difference (in terms of the RL components of each) between the chess and cart-pole examples and the grid-world example discussed previously: the size of the state-space.  

- With the chess example: the number of possible configurations of chess pieces on the board is enormous - i.e., the number of states or size of state space - is on the order of $10^{120}$.  That is far larger than the number of atoms in the known universe!


- With the cart-pole example: each of the four descriptors of the enviroment is continuous (or highly discretized) - thus the number of states is infinite.  


In both cases the (very large) size of the state space presents practical problems (while the size of the action space is not a problem).  Why?  Because rememver that in order to learn the $Q$ function we have to be able to cycle through *every possible state*.  Practically speaking then even a state space of size $10^{120}$ is far too large for us to be able to learn a representative $Q$ effectively.  

Even more practically, remember that so far we have been storing the $Q$ function as a matrix of size $N \times M$, where $N$ and $M$ are the sizes of the state and action spaces respectively.  But the size of $N$ is - for these two problems and for most real world problems (in automatic control and game AI) far too large store in memory!  

In sum the problem here that most real RL problems the **size of the state space** makes computing and storing the $Q$ function/matrix impossible.  Some RL problems can also have issues with large sized action spaces, but this is typically not as problematic (as action spaces can be more easily discretized).

## 4.1  Compactly representing state space of the $Q$ function

We need a way to compactly represent *the vertical or state-space dimension* of $Q$ when we have a very large (and possibly infinite number) of states.  Instead of trying to compute and record every input/output pair - which is impossible once the input space is too large or infinite - we can try the next best thing: represent $Q$ as a *parameterized function* and simply fit it to the input/output data.

Does the concept of "fitting a parameterized function to a set of input/output data" sound familiar?  Thats just a fancy way of saying that we'll use **regression** - a fundamental machine learning tool.  

## 4.2  Using regression to represent the columns of Q

Instead of storing the entire matrix $Q$, we will substitute a parameterized function for each of its columns - independently.   The $k^{th}$ available action $a_k$ has a corresponding slice of $Q$ (an $N \times 1$ column vector)

$Q_k=\left[\begin{array}{c}
Q\left(f(\sigma_{0}),\alpha_{k}\right)\\
Q\left(f(\sigma_{1}),\alpha_{k}\right)\\
\vdots\\
Q\left(f(\sigma_{N}),\alpha_{k}\right)
\end{array}\right]$

As with the $Q$ matrix itself - the length of this vector is far too long to completely compute or store.  Note too that with the action fixed at $\alpha_{k}$ every entry of this column is a single *point*, an input/output pair.  For example the $i^{th}$ point is $(f(\sigma_{i}),Q\left(f(\sigma_{i}),\alpha_{k}\right))$.  

So in other words, to represent the column of values using a parameterized function $h$ needs to be a function of the input here $f(s)$.  For example, suppose we have $P$ features - i.e., that  $f$ is a $P$ length vector $f(s) = (~f_1(s)~,f_2(s)~,...,f_P(s)~)$ then the simplest - but still commonly used - parameterized representation is a linear combination of the input features


$h_{\theta_k}(s) = \theta_{1}\,f_{1}\left(s\right)+\theta_{2}\,f_{2}\left(s\right)+\cdots+\theta_{P}\,f_{P}\left(s\right)$

In the very simplest instance each feature here is just a variable of the state (an entry of the vector $s$), so this is a linear combination of the entries of the state vector.  

However this combination can be quite complicated.  The features may be real quantities derived from the states themselves and - since this is a regression problem - we can employ any **mathematical basis** we wish e.g., polynmoial, Fourier, decision trees, neural networks making the above highly nonlinear.  In this case each $f_p$ can be a parameterized basis function like e.g., a neural network basis element.

But regardless of the basis / feature type chosen the hope here is that - when properly tuned - that this model accurately represents what would be the true $Q$ values as

$h_{\theta_k}(s) \approx Q\left(f(s),\alpha_{k}\right))$.

While we now need to tune these parameters, the benefit is quite plain: instead of storing the original column vector we need only store the parameter values - e.g., in the simplest case of a linear approximator we have only $P$ values. 

Finally note that since $Q$ originally has $M$ columns and we are replacing each with a parameterized model, we will $M$ total parameterized models representing $Q$.  These models are almost always taken to have the same form (i.e., all are either linear, neural networks, kernels, etc.,) so we can distinguish them via their index $h_{\theta_0}(s)~,h_{\theta_1}(s),\cdots~h_{\theta_M}(s)$.

## 4.3  The Q-learning update step

Remember that with the original Q-learning update discussed in Section 2.2 we update a single entry of the matrix at each step as

$Q\left(s_{k-1},\,a_{k}\right)=r_{k}+\gamma\cdot\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{k},\,a_{i}\right)$


Now, however, we have a parameterized function taking the place of each column of $Q$, and because of this we need to split this step *into two pieces* when adapting it to our current situation.  

In the first step we generate the update quantity.  

$q_{k}=r_{k}+\gamma\cdot\underset{i=1...M}{\text{maximum}}\,\,h_{\theta_i}(s_k)$


This is precisely what we did previously - the only thing that has changed is that now our $Q$ matrix is represented by a list of parameterized functions - one for each column - and so the *maximum* in the above update step is over our functions (and not the literal columns of a matrix).  Because we have no $Q$ matrix of values we call the value on the left hand side $q_{k}$.

But we also need to update our representation of $Q$ with this data point.  In the original case, where we had no function approximators, we simply replaced the entry from the appropriate column of the $Q$ matrix with this value.  The analog here is that we now need to *re-tune* the parameters of the parameterized function taking the place of the appropriate column of $Q$.  

The best way to do this is to refit the appropriate model to this datapoint via solving a regularized Least Squares problem.  Letting $j$ denote the index of the maximum acheived on the right hand side of the update above, an unregularized Least Squares would mean finding the parameters $\theta_j$ that minimize the quantity $(h_{\theta_j}(s_k)-q_{k})^{2}$.  However minimizing this alone would overfit our parameters to the single point $q_k$, and we would lose all of the information contained in the previously tuned parameters.  In order to avoid this we can add a simple regularizer that ensures that the new parameters aren't too different from the previous ones: denoting the previous set of parameters $\theta_{j,\text{previous}}$ we add an appropriate regularizer to the Least Squares

$\underset{\theta_{j}}{\text{minimize}}\,\,\left(h_{\theta_{j}}\left(s_{k}\right)-q_{k}\right)^{2}+\beta\left\Vert \theta_{j}-\theta_{j,\text{previous}}\right\Vert _{2}^{2}$


Here $\beta$ is a parameter we can tune to ensure that the new parameters are either very close to the previously calculated parameters (by setting $\beta$ to a large positive number), or lighten this restriction by setting $\beta$ low.  Typically this is either set to be a small constant, or increased gradually at each episode of simulation.

Regardless of the $\beta$ setting, and the form of $h$, this regularized Least Squares problem has a closed form solution.  Setting the gradient in $\theta_j$ above to zero and solving gives the solution

$\theta_{j}=\theta_{j,\text{previous}}-\frac{1}{\beta}(h_{\theta_{j}}\left(s_{k}\right)-q_{k})\nabla h_{\theta_{j}}\left(s_{k}\right)$

Due to the form of this update, this is often interpreted as a stochastic gradient descent step.

## 4.4 An algorithm for Q learning with function approximators

All together a basic algorithm for Q learning with function approximators looks like

------
Initialize parameters $\theta_1,...,\theta_M$, choose value for $\gamma \in [0,1], and a step size rule$

**for** e = 1...E (the maximum number of episodes)
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Select a random initial state $s_0$ and $k=0$
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **while** end state OR maximum iteration count NOT reached

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;select a random action $a_k$ at our current state $s_{k-1}$ and move to $s_k$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;update $Q$ as 
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$q_{k}=r_{k}+\gamma\cdot h_{\theta_{j}}$ where $h_{\theta_{j}}= \underset{i=1...M}{\text{maximum}}\,\,h_{\theta_i}(s_k)$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\theta_{j}=\theta_{j,\text{previous}}-\frac{1}{\beta}(h_{\theta_{j}}\left(s_{k}\right)-q_{k})\nabla h_{\theta_{j}}\left(s_{k}\right)$
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$k \longrightarrow k+1$

-----

Due to the randomness involved in choosing states this procedure can produce results that vary significantly in quality from one run to the next.  Because of this one typically runs this method a number of times, saving the set of parameters associated with the best run.  

We use this procedure to train an agent for the cart-pole problem in the Python cell below. When you run this cell the first few episodes of training will be animated - much as we did with the grid world example in Section 2.

In [1]:
from my_cartpole import my_cartpole
test = my_cartpole()
test.qlearn(gamma=0.8,num_episodes = 100)

[2016-12-06 12:32:26,885] Making new env: CartPole-v0


q-learning process complete, best number of average steps in test runs = 462.5


You can expose the Q learning function here by activating the Python cell below - it maches the pseudo-code given above.

In [3]:
test.qlearn??

## 4.4  What is the best action per state given a learned Q function?

Given the parameterized functions representing $Q$ are learned properly, how should the agent move at each state?  Precisely as we did when with problems like grid world - where we chose the action maximizing the corresponding $Q$ value.  The only difference here is that each column of $Q$ is a parameterized function.

So at a given state $s$ the action $a_j$ providing this maximimum is given as the argument of this maximum

$j=\underset{i=1...M}{\text{argmax}}\,\,h_{\theta_i}(s_k)$

This is precisely how the learned agent controls the cart-pole, as you can see by activating the Python cell below.  

In [ ]:
# run a few test runs of cart-pole using the trained function approximators
test.animate_test_run()
test.animate_test_run()

To expose the controller activate the Python cell below.

In [4]:
test.animate_test_run??

## 5. Next version to dos:

I.  General discussion of when using RL is worth using - vs engineering things by hand

- the cost of simulation + cost of RL feature engineering + RL algorithm engineering vs cost of hand engineering solutions


II.  Examples using nonlinear function approximators

- fourier basis
- deep nets / conv nets
    
III. Section on Implementation tips and tricks

Here we can talk about the many engineering choices one has to speed up learning when of an RL algorithm like Q learning with function approximators.  e.g.,

- step length selection

- exploration / exploitation trade-off

- memory replay

- etc.,